In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 4

In [2]:
train = pd.read_csv('../data/train.csv')
print(train.columns)
print('Unique locations: %d' % len(train['location'].unique()))
train.head()

Index(['id', 'location', 'fault_severity'], dtype='object')
Unique locations: 929


,id,location,fault_severity
0,14121,location 118,1
1,9320,location 91,0
2,14394,location 152,1
3,8218,location 931,1
4,14804,location 120,0


In [87]:
test = pd.read_csv('../data/test.csv')
print(len(test))
test.describe()

11171


,id
count,11171.000000
mean,9284.330230
std,5360.065369
min,2.000000
25%,4630.000000
50%,9316.000000
75%,13926.000000
max,18552.000000


In [3]:
event_type = pd.read_csv('../data/event_type.csv')
event_type.head()

,id,event_type
0,6597,event_type 11
1,8011,event_type 15
2,2597,event_type 15
3,5022,event_type 15
4,5022,event_type 11


In [84]:
log_feature = pd.read_csv('../data/log_feature.csv')
print('Unique features: %d' % len(log_feature['log_feature'].unique()))
# print(log_feature.head())
sparse_feature = log_feature.groupby(['id', 'log_feature']).max()
sparse_feature = sparse_feature.unstack(1).fillna(0)
sparse_feature = sparse_feature['volume'].reset_index()
sparse_feature

Unique features: 386


log_feature,id,feature 1,feature 10,feature 100,feature 101,feature 102,feature 103,feature 104,feature 105,feature 106,...,feature 90,feature 91,feature 92,feature 93,feature 94,feature 95,feature 96,feature 97,feature 98,feature 99
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,0,0,0,0,0,0,0,33,0,...,0,0,0,0,0,0,0,0,0,0
9,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
resource_type = pd.read_csv('../data/resource_type.csv')
resource_type.head()

,id,resource_type
0,6597,resource_type 8
1,8011,resource_type 8
2,2597,resource_type 8
3,5022,resource_type 8
4,6852,resource_type 8


In [5]:
severity_type = pd.read_csv('../data/severity_type.csv')
severity_type.head()

,id,severity_type
0,6597,severity_type 2
1,8011,severity_type 2
2,2597,severity_type 2
3,5022,severity_type 1
4,6852,severity_type 1


## Join all dataframes

In [86]:
df = train.merge(sparse_feature, on='id')
# df = train.merge(gs, on='id')
# df = df.merge(event_type, on='id')
# df = df.merge(resource_type, on='id')
# df = df.merge(severity_type, on='id')
print('Train count: %d' % len(train))
print('df count: %d' % len(df))
df.head()

Train count: 7381
df count: 7381


,id,location,fault_severity,feature 1,feature 10,feature 100,feature 101,feature 102,feature 103,feature 104,...,feature 90,feature 91,feature 92,feature 93,feature 94,feature 95,feature 96,feature 97,feature 98,feature 99
0,14121,location 118,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9320,location 91,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14394,location 152,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8218,location 931,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14804,location 120,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
